In [ ]:
mount = '/content/drive'
from google.colab import drive
drive.mount(mount)

# Git

In [2]:
from getpass import getpass
token = getpass("GitHub token: ")

GitHub token: ··········


In [3]:
%pwd

'/content'

In [4]:
%cd /content/drive/MyDrive/MusicNBrain2026-Spring/MusicNBrain-RealtimeAvatar

/content/drive/MyDrive/MusicNBrain2026-Spring/MusicNBrain-RealtimeAvatar


In [5]:
!git pull origin main

From https://github.com/YueMenaYang/MusicNBrain-RealtimeAvatar
 * branch            main       -> FETCH_HEAD
Already up to date.


In [6]:
!git config --global user.email "yq171014@columbia.edu"
!git config --global user.name "YueMenaYang"

In [6]:
!git add .
!git commit -m "Update audio2audio to audio2audio_musicOnly"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@c0890022ed05.(none)')


In [21]:
!git push origin main

Everything up-to-date


# Code

In [7]:
!pip -q install -U google-genai soundfile
!apt-get -qq install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.7/724.7 kB 24.3 MB/s eta 0:00:00


In [8]:
from google.colab import auth
auth.authenticate_user()

from google import genai
import os
import asyncio, time
import numpy as np
import soundfile as sf
from pathlib import Path
from google.genai.types import Content, Part, LiveConnectConfig


In [23]:
AUDIO_INP_PATH = "/content/drive/MyDrive/MusicNBrain2026-Spring/inputs/piano1.m4a"
PROJECT_ID = "musicrealtimeavatar"
LOCATION = "us-east4"
MODEL_ID = "gemini-live-2.5-flash-native-audio"
# AUDIO_OUT_PATH = "/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_c.wav"

# PCM
PCM_RATE_IN = 16000
PCM_RATE_OUT = 24000


client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [24]:
# Quick Test Connection

# from google.genai.types import LiveConnectConfig, Content, Part

# MODEL_ID = "gemini-live-2.5-flash-native-audio"

# config = LiveConnectConfig(
#     response_modalities=["AUDIO"],
#     system_instruction=(
#         "You are a supportive music teacher. "
#         "Listen to the performance audio and give concise, actionable feedback. "
#         "Keep the feedback under 90 seconds."
#     ),
# )

# import asyncio

# async def connect_smoketest():
#     async with client.aio.live.connect(model=MODEL_ID, config=config) as session:
#         await session.send_client_content(
#             turns=Content(role="user", parts=[Part(text="Say hello in audio.")])
#         )
#         async for msg in session.receive():
#             if msg.server_content and msg.server_content.turn_complete:
#                 break
#     return "OK"

# await connect_smoketest()

In [25]:
import subprocess
from pathlib import Path

in_audio = Path(AUDIO_INP_PATH)
assert in_audio.exists(), f"File not found: {in_audio}"

# pcm path (input)
pcm_path = in_audio.with_suffix("")
pcm_path = pcm_path.parent / f"{pcm_path.name}_16k_mono.pcm"

# output path
project_root = in_audio.parent.parent
outputs_dir = project_root / "outputs"
outputs_dir.mkdir(exist_ok=True)
AUDIO_OUT_PATH = outputs_dir / f"{in_audio.stem}_c.wav"

In [26]:
# in_audio = Path(AUDIO_INP_PATH)
# assert in_audio.exists(), f"File not found: {in_audio}"

# pcm_path = in_audio.with_suffix("")
# pcm_path = pcm_path.parent / f"{pcm_path.name}_16k_mono.pcm"

subprocess.run([
    "ffmpeg", "-y",
    "-i", str(in_audio),
    "-ac", "1",
    "-ar", "16000",
    "-f", "s16le",
    str(pcm_path)
], check=True)

print("WAV:", in_audio)
print("PCM:", pcm_path, "bytes =", pcm_path.stat().st_size)


WAV: /content/drive/MyDrive/MusicNBrain2026-Spring/inputs/piano1.m4a
PCM: /content/drive/MyDrive/MusicNBrain2026-Spring/inputs/piano1_16k_mono.pcm bytes = 6360406


In [27]:
def chunk_bytes(b: bytes, chunk_size: int):
    for i in range(0, len(b), chunk_size):
        yield b[i:i+chunk_size]

async def stream_pcm_realtime_and_get_commentary_concurrent(
    pcm_file: str,
    out_wav: str = "/content/commentary.wav",
    chunk_ms: int = 100,
    extra_jitter_ms: int = 0,
    print_progress: bool = True,
):

    t_start = time.perf_counter()
    t_first_send = None
    t_first_audio = None
    t_turn_complete = None

    config = LiveConnectConfig(
        response_modalities=["AUDIO"],
        system_instruction=(
            "You are a supportive music teacher. "
            "Listen to the performance audio and give concise, actionable feedback. "
            "Focus on rhythm and tempo stability, dynamics, articulation, and practice tips. "
            "Be detailed. "
            "Speak slowly, naturally and keep it under 90 seconds."
        ),
    )

    pcm_bytes = Path(pcm_file).read_bytes()
    seconds_per_chunk = chunk_ms / 1000.0
    chunk_size = int(PCM_RATE_IN * seconds_per_chunk) * 2  # int16 -> 2 bytes

    collected_out_pcm = bytearray()
    received_turn_complete = asyncio.Event()

    async with client.aio.live.connect(model=MODEL_ID, config=config) as session:
        # Send initial instruction
        await session.send_client_content(
            turns=Content(role="user", parts=[Part(text="Give coaching feedback on this performance audio.")]),
            turn_complete=True
        )

        async def receiver():
            """Continuously receive server messages to avoid keepalive timeout."""
            nonlocal t_first_audio, t_turn_complete
            try:
                async for msg in session.receive():
                    if msg.data:
                        if t_first_audio is None:

                          t_first_audio = time.perf_counter()

                        collected_out_pcm.extend(msg.data)
                    if msg.server_content and msg.server_content.turn_complete:

                        t_turn_complete = time.perf_counter()

                        received_turn_complete.set()
                        break
            except Exception as e:
                # If receiver fails, propagate by setting event so sender can stop
                received_turn_complete.set()
                raise e

        async def sender():
            """Stream audio at real-time pace."""
            nonlocal t_first_send
            start_wall = time.time()
            sent_chunks = 0
            try:
                for chunk in chunk_bytes(pcm_bytes, chunk_size):
                    if received_turn_complete.is_set():
                        break

                    t0 = time.time()
                    if t_first_send is None:
                      t_first_send = time.perf_counter()
                    await session.send_realtime_input(
                        audio={"data": chunk, "mime_type": "audio/pcm;rate=16000"}
                    )
                    sent_chunks += 1

                    # pacing
                    target = seconds_per_chunk + extra_jitter_ms / 1000.0
                    elapsed = time.time() - t0
                    if target - elapsed > 0:
                        await asyncio.sleep(target - elapsed)

                    if print_progress and sent_chunks % 10 == 0:
                        streamed_s = sent_chunks * seconds_per_chunk
                        wall_s = time.time() - start_wall
                        print(f"Streamed ~{streamed_s:.1f}s in {wall_s:.1f}s wall time")

                # End-of-stream
                await session.send_realtime_input(audio_stream_end=True)

                # Wait for the model to finish the turn (with a generous timeout)
                await asyncio.wait_for(received_turn_complete.wait(), timeout=120)

            except asyncio.TimeoutError:
                # Even if the model didn’t finish, we’ll save what we have
                print("Warning: model turn did not complete before timeout; saving partial audio.")
            except Exception as e:
                print("Sender error:", repr(e))
                raise

        # Run sender + receiver concurrently
        await asyncio.gather(receiver(), sender())

    # Save collected PCM -> WAV
    audio_i16 = np.frombuffer(collected_out_pcm, dtype=np.int16)
    sf.write(out_wav, audio_i16, PCM_RATE_OUT, subtype="PCM_16")

    t_end = time.perf_counter()

    metrics = {
        "mode": "single",
        "ttfb_s": None if t_first_audio is None else (t_first_audio - t_first_send),
        "turn_complete_s": None if t_turn_complete is None else (t_turn_complete - t_start),
        "total_wall_s": t_end - t_start,
    }

    print("SINGLE LATENCY METRICS:")
    for k, v in metrics.items():
        print(f"{k}: {v:.3f}" if isinstance(v, float) else f"{k}: {v}")

    return out_wav

In [28]:
out_path = await stream_pcm_realtime_and_get_commentary_concurrent(
    pcm_file=str(pcm_path),
    out_wav= AUDIO_OUT_PATH,
    chunk_ms=100,          # try 40 or 20 later
    extra_jitter_ms=0,     # try 10~30 to simulate jitter
    print_progress=True
)

print("Saved:", out_path)


Streamed ~1.0s in 1.0s wall time
Streamed ~2.0s in 2.0s wall time
Streamed ~3.0s in 3.0s wall time
SINGLE LATENCY METRICS:
mode: single
ttfb_s: 0.547
turn_complete_s: 4.474
total_wall_s: 4.964
Saved: /content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_c.wav


In [29]:
from IPython.display import Audio, display
display(Audio(AUDIO_OUT_PATH))

## Progressive Segments


In [30]:
def chunk_bytes(b: bytes, chunk_size: int):
    for i in range(0, len(b), chunk_size):
        yield b[i:i+chunk_size]

async def stream_and_save_progressive_segments(
    pcm_file: str,
    out_dir: str = "/content/commentary_segments",
    chunk_ms: int = 100,
    extra_jitter_ms: int = 0,
    segment_seconds: float = 5.0,
    print_progress: bool = True,
):
    os.makedirs(out_dir, exist_ok=True)

    t_start = time.perf_counter()
    t_first_send = None
    t_first_audio = None
    t_first_segment = None

    config = LiveConnectConfig(
        response_modalities=["AUDIO"],
        system_instruction=(
            "You are a supportive music teacher. "
            "Listen to the performance audio and give concise, actionable feedback. "
            "Focus on rhythm and tempo stability, dynamics, articulation, and practice tips. "
            "Be detailed. "
            "Speak slowly, naturally and keep it under 90 seconds."
        ),
    )

    pcm_bytes = Path(pcm_file).read_bytes()
    seconds_per_chunk = chunk_ms / 1000.0
    send_chunk_size = int(PCM_RATE_IN * seconds_per_chunk) * 2  # int16 -> 2 bytes

    # For segmenting output audio
    seg_samples_target = int(PCM_RATE_OUT * segment_seconds)
    seg_buffer = np.zeros((0,), dtype=np.int16)

    segment_paths = []
    received_turn_complete = asyncio.Event()

    async with client.aio.live.connect(model=MODEL_ID, config=config) as session:
        await session.send_client_content(
            turns=Content(role="user", parts=[Part(text="Give coaching feedback on this performance audio.")]),
            turn_complete=True
        )

        async def receiver():
            nonlocal seg_buffer, t_first_audio, t_first_segment
            seg_idx = 1
            received_samples_total = 0

            async for msg in session.receive():
                if msg.data:
                    if t_first_audio is None:
                        t_first_audio = time.perf_counter()
                    chunk_i16 = np.frombuffer(msg.data, dtype=np.int16)
                    if chunk_i16.size > 0:
                        seg_buffer = np.concatenate([seg_buffer, chunk_i16])
                        received_samples_total += chunk_i16.size

                        # Write out segments whenever we have enough
                        while seg_buffer.size >= seg_samples_target:
                            seg = seg_buffer[:seg_samples_target]
                            seg_buffer = seg_buffer[seg_samples_target:]

                            seg_path = f"{out_dir}/commentary_seg_{seg_idx:03d}.wav"
                            sf.write(seg_path, seg, PCM_RATE_OUT, subtype="PCM_16")
                            segment_paths.append(seg_path)

                            now = time.perf_counter()
                            if t_first_segment is None:
                                t_first_segment = now

                            if print_progress:
                                dur_s = seg_samples_target / PCM_RATE_OUT
                                print(f"[OUT] wrote segment {seg_idx:03d} (~{dur_s:.1f}s): {seg_path}")

                            seg_idx += 1

                if msg.server_content and msg.server_content.turn_complete:
                    received_turn_complete.set()
                    break

        async def sender():
            nonlocal t_first_send
            start_wall = time.time()
            sent_chunks = 0

            for chunk in chunk_bytes(pcm_bytes, send_chunk_size):
                # If model already ended (rare), stop sending
                if received_turn_complete.is_set():
                    break

                t0 = time.time()
                if t_first_send is None:
                    t_first_send = time.perf_counter()
                await session.send_realtime_input(
                    audio={"data": chunk, "mime_type": "audio/pcm;rate=16000"}
                )
                sent_chunks += 1

                target = seconds_per_chunk + extra_jitter_ms / 1000.0
                elapsed = time.time() - t0
                if target - elapsed > 0:
                    await asyncio.sleep(target - elapsed)

                if print_progress and sent_chunks % 10 == 0:
                    streamed_s = sent_chunks * seconds_per_chunk
                    wall_s = time.time() - start_wall
                    print(f"[IN ] streamed ~{streamed_s:.1f}s in {wall_s:.1f}s wall time")

            await session.send_realtime_input(audio_stream_end=True)

            # wait for model completion
            try:
                await asyncio.wait_for(received_turn_complete.wait(), timeout=180)
            except asyncio.TimeoutError:
                print("Warning: model did not complete turn before timeout; saving partial output.")

        # Run both concurrently (prevents keepalive timeout)
        await asyncio.gather(receiver(), sender())

    # Flush any remaining partial audio as a final segment
    if seg_buffer.size > 0:
        seg_path = f"{out_dir}/commentary_seg_{len(segment_paths)+1:03d}_tail.wav"
        sf.write(seg_path, seg_buffer, PCM_RATE_OUT, subtype="PCM_16")
        segment_paths.append(seg_path)

        if print_progress:
            print(f"[OUT] wrote tail segment: {seg_path}")

    t_end = time.perf_counter()

    metrics = {
    "mode": "progressive",
    "ttfb_s": None if (t_first_audio is None or t_first_send is None) else (t_first_audio - t_first_send),
    "ttfs_s": None if (t_first_segment is None or t_first_send is None) else (t_first_segment - t_first_send),
    "total_wall_s": t_end - t_start,
    }

    print("PROGRESSIVE LATENCY METRICS:")
    for k, v in metrics.items():
        print(f"{k}: {v:.3f}" if isinstance(v, float) else f"{k}: {v}")

    return segment_paths

In [31]:
segment_paths = await stream_and_save_progressive_segments(
    pcm_file=str(pcm_path),          # from your earlier conversion cell
    out_dir="/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments",
    chunk_ms=100,
    extra_jitter_ms=0,
    segment_seconds=5.0,             # try 1.0 for more “live”
    print_progress=True
)

print("\nTotal segments:", len(segment_paths))
segment_paths[:5]


[IN ] streamed ~1.0s in 1.0s wall time
[OUT] wrote segment 001 (~5.0s): /content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_001.wav
[IN ] streamed ~2.0s in 2.3s wall time
[OUT] wrote segment 002 (~5.0s): /content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_002.wav
[IN ] streamed ~3.0s in 3.7s wall time
[OUT] wrote segment 003 (~5.0s): /content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_003.wav
[IN ] streamed ~4.0s in 4.9s wall time
[OUT] wrote tail segment: /content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_004_tail.wav
PROGRESSIVE LATENCY METRICS:
mode: progressive
ttfb_s: 0.566
ttfs_s: 2.090
total_wall_s: 6.263

Total segments: 4


['/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_001.wav',
 '/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_002.wav',
 '/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_003.wav',
 '/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_004_tail.wav']

In [32]:
from IPython.display import Audio, display

for p in segment_paths:
    print(p)
    display(Audio(p))


/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_001.wav


/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_002.wav


/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_003.wav


/content/drive/MyDrive/MusicNBrain2026-Spring/outputs/piano1_segments/commentary_seg_004_tail.wav
